### NE extraction: low computational cost but runs for hours (2h-3h?)
- NER (with Spacy / StanfordNLP) based on all evidence
- processed data is ./data/spacy_ner_tot.json
    - It is a match dictionary from NE to evidence
    - e.g. "New York City": ["evidence-0000", "evidence-00000"]

In [1]:
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize
# from transformers import BertTokenizer
import pandas as pd
from collections import defaultdict
import json
import spacy
import os
import string

EVI_PATH = "./data/evidence.json"
PATH_TO_JAR = "../../software/stanford_ner/stanford-ner.jar"
PATH_TO_MODEL = "../../software/stanford_ner/classifiers/english.all.3class.distsim.crf.ser.gz"
FILE_PRE = "./data/spacy_ner/ner_match_"
nlp = spacy.load("en_core_web_sm")

ner_tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL,path_to_jar=PATH_TO_JAR, encoding='utf-8')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
puncu_remove = str.maketrans('', '', string.punctuation)


2023-05-14 21:06:38.246810: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 21:06:38.462509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 21:06:39.491627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-14 21:06:40.554926: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:06:40.581

In [2]:
evi_df = pd.read_json(EVI_PATH, orient="index").reset_index().rename({0: "evidence", "index": "evi_id"}, axis=1)

In [3]:
ner_map = defaultdict(lambda: defaultdict(list))

for id, evidence in evi_df.values[:1000]:
    doc = nlp(evidence)
    for ent in doc.ents:
        ner_map[ent.label_][ent.text.lower()].append(id)


# total_epo: 1208.827
START_EPO = 0
END_EPO = 1210
FILE_NO = 4
BATCH_SIZE = 1000

# Loop through evidence
ner_map = defaultdict(lambda: defaultdict(list))
for epoch in range(START_EPO, END_EPO):
    cur_data = evi_df.loc[epoch*BATCH_SIZE:(epoch+1)*BATCH_SIZE].values
    
    ## NE extraction
    for id, evidence in cur_data:
        doc = nlp(evidence)
        for ent in doc.ents:
            ner_map[ent.label_][ent.text.lower().translate(puncu_remove)].append(id)
    
    ## save file
    if epoch%50 == 0:
        file_name = FILE_PRE + str(FILE_NO) + ".json"
        with open(file_name, 'w') as f:
            json.dump(ner_map, f)
        print(f"File saved: {file_name}, epoch: {epoch}")
    
## save work
file_name = FILE_PRE + str(FILE_NO) + ".json"
with open(file_name, 'w') as f:
    json.dump(ner_map, f)
print(f"File saved: {file_name}")


File saved: ./data/spacy_ner/ner_match_4.json, epoch: 0
File saved: ./data/spacy_ner/ner_match_4.json, epoch: 50
File saved: ./data/spacy_ner/ner_match_4.json, epoch: 100


## Combine files

In [10]:
# Combine saved file

parent_dir = "./data/spacy_ner/"


file_names = os.listdir(parent_dir)


print(file_names)

total_dict = defaultdict(lambda: defaultdict(list))
for file_name in file_names[0:]:
    with open(parent_dir + file_name, "r") as cur_file:
        cur_dict = json.loads(cur_file.read())
    for ner_key in cur_dict.keys():
        for key in cur_dict[ner_key].keys():
            total_dict[ner_key][key]+=cur_dict[ner_key][key]

file_name = f"./data/spacy_ner_tot.json"
with open(file_name, 'w') as f:
    json.dump(ner_map, f)
print(f"File saved: {file_name}")